# Predicción de la producción de activos industriales utilizando el algoritmo de detección de anomalías Holt-Winters+Brutlag

### 1. Cargar librerías

In [1]:
import pandas as pd
import numpy as np

from functools import reduce
import warnings
import time

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error

### 2. Lectura de datos y preparación

In [2]:
Y_train=pd.read_csv(('Datos/train_y.csv'))
Y_train.head()

,SAMPLE_ID,PRODUCTION_GROUP_2,PRODUCTION_GROUP_3
0,1,3850220.0,9019860.320
1,2,3793240.0,8568867.304
2,3,3870570.0,8549258.912
3,4,3801380.0,8451216.952
4,5,3752540.0,8323762.404


In [3]:
data=pd.read_csv('Output/X_train_formateado.csv')  #X_train_formateado.csv se obtiene mediante el código 
                                                   #del notebook 0.Exploración y preparación de datos
data.head(3)

,asset_id,group_id,week,weekday,t1,t2,t3,t4,total_t,week_day,date,day
0,1,2,1,1,NaN,NaN,19.0,0.0,2,1_1,2018-01-01,1
1,1,2,1,2,NaN,NaN,8.0,0.0,2,1_2,2018-01-02,2
2,1,2,1,3,NaN,NaN,9.0,0.0,2,1_3,2018-01-03,3


In [4]:
#Leer datos de capacidad nominal por asset por semana

capacity=pd.read_csv('Datos/assets.csv', header=0, names=['asset_id', 'nominalc_week'],
                     dtype={'asset_id':int, 'nominalc_week':float} )

#agregar columna de capacidad por día
capacity['nominalc_day']=capacity['nominalc_week']/7     
capacity.head()

,asset_id,nominalc_week,nominalc_day
0,1,132000.0,18857.142857
1,2,93000.0,13285.714286
2,3,238600.0,34085.714286
3,4,206000.0,29428.571429
4,5,310000.0,44285.714286


In [5]:
data=data.merge(capacity[['asset_id','nominalc_day']])
data.head()

,asset_id,group_id,week,weekday,t1,t2,t3,t4,total_t,week_day,date,day,nominalc_day
0,1,2,1,1,NaN,NaN,19.0,0.0,2,1_1,2018-01-01,1,18857.142857
1,1,2,1,2,NaN,NaN,8.0,0.0,2,1_2,2018-01-02,2,18857.142857
2,1,2,1,3,NaN,NaN,9.0,0.0,2,1_3,2018-01-03,3,18857.142857
3,1,2,1,4,NaN,NaN,6.0,0.0,2,1_4,2018-01-04,4,18857.142857
4,1,2,1,5,NaN,NaN,6.0,2.0,2,1_5,2018-01-05,5,18857.142857


In [6]:
#Crear objeto GroupBy de Pandas para iterar despues en las tablas de cada asset
data=data.set_index('asset_id')
porAsset=data.groupby(data.index)

### 3. Definición de funciones

In [7]:
 
def ClasificacionDeObservaciones(df):  
    '''FUNCIÓN PARA CLASIFICAR ANOMALÍAS MEDIANTE LOS ALGORITMOS HOLT-WINTER Y BRUTLAG PARA CADA UNA 
    DE LAS SERIES CORREPONDIENTES A LAS MEDICIONES HECHAS EN CADA FÁBRICA
    df: pandas data frame
    gamma_HW: default True para usar el gamma obtenido de Holt_Winters, si es False se deberá establecer
    el valor de gamma
    gamma: valor de gamma para clasificar anomalias'''
    outliers_table = {}
    PERIOD = 7 
    if df.total_t.mean() == 2:
        mediciones=[3,4]
    else:
        mediciones=[1,2,3,4]
    for x in mediciones:  #para cada una de las mediciones
        t='t'+str(x)
    #Entrenar modelo con Holt-Winters
        model=ExponentialSmoothing(df[t], trend='add', seasonal='add', seasonal_periods=7, freq="D").fit()         
        
    #Brutalg Algorithm     
        # 1. Cálculo de los márgenes de confianza mediante el algoritmo brutlag
        prediction = model.predict(start=df[t].index[0], end=df[t].index[-1])  #predicción con Holt-Winters
        GAMMA = model.params['smoothing_seasonal']   #gamma calculado por Holt-Winters 

        M = 2           # toma un valor entre 2 y 3 (ver referencia 3)   
        UB = []         # margen de confianza superior 
        LB = []         # margen de confianza inferior
        dt = []
        for i in range(len(prediction)):
            diff = df[t][i]-prediction[i]
            if i < PERIOD:
                dt.append(GAMMA*abs(diff))
                UB.append(prediction[i]+M*dt[i])
                LB.append(prediction[i]-M*dt[i])
            else:
                dt.append(GAMMA*abs(diff) + (1-GAMMA)*dt[i-1])
                UB.append(prediction[i]+M*dt[i-PERIOD])
                LB.append(prediction[i]-M*dt[i-PERIOD])

        # 2 Clasificacion de las observaciones como normales o anómalas
        outlier = []
        outlier_date = []
        
        for i in range(len(df[t])):
            outlier_date.append(df.index[i])
            if (UB[i] <= df[t][i])  and i > PERIOD:
                outlier.append(1)
            else:
                outlier.append(0)
        outliers_table['outlier_'+t]=pd.DataFrame({"date": outlier_date, "outlier_"+t: outlier, "gamma_"+t:GAMMA})
    clasificacion_observaciones = reduce(lambda  left,right: pd.merge(left,right,on=['date'], 
                                                                      how='left'),outliers_table.values()).set_index('date')
    new_df=pd.merge(df,clasificacion_observaciones, left_index=True, right_index=True)
    return  new_df
       

In [12]:
def resultados(dict_df):
    '''FUNCION PARA CALCULAR EL ERROR ASIGNANDO EL VALOR DE PRODUCCION NOMINAL DIARIO A LOS DÍAS EN QUE NO HUBO 
    NINGUNA ANOMALÍAS Y 0 EN CASO CONTRARIO 
    
    dict_df: diccionario resultante con la detección de anomalías con un dataframe por cada asset

    '''
    
    X=pd.concat(dict_df.values())
    X=X.reset_index(drop=True)
    
    X=X[['asset_id', 'group_id', 'week', 'weekday','total_t','nominalc_day', 'outlier_t1','outlier_t2','outlier_t3','outlier_t4']]
    

    X['produccion']=np.where(
        ( ( X['outlier_t1']==1 ) | ( X['outlier_t2']==1 ) | (X['outlier_t3']==1)  | ( X['outlier_t4']==1 ) ),
        0, X['nominalc_day'] )
   
    #Agregamos los datos por grupo y semana para obtener Y_predicted

    ProduccionGrupoSemana=X.groupby(['group_id','week'])
    Y_predicted=ProduccionGrupoSemana.agg({'produccion':'sum'})
    Y_predicted=Y_predicted.reset_index()

    # Configuramos el formato  Y_predicted para tener el mismo que en Y_train
    Y_predicted=pd.pivot_table(Y_predicted,index=['week'], columns=['group_id'], values=['produccion'])
    Y_predicted=Y_predicted.reset_index()
    Y_predicted.columns.set_levels(['production_group_2', 'production_group_3', ''], level=1, inplace=True)
    Y_predicted.columns=list(map("".join, Y_predicted))
    Y_predicted.columns=Y_train.columns

    # Calculamos el error para cada guropo
    MSE2=mean_squared_error(Y_train.PRODUCTION_GROUP_2, Y_predicted.PRODUCTION_GROUP_2)
    MSE3=mean_squared_error(Y_train.PRODUCTION_GROUP_3, Y_predicted.PRODUCTION_GROUP_3)

    
    #Calculamos el error total
    Y_train_tot = Y_train.PRODUCTION_GROUP_2.to_list() + Y_train.PRODUCTION_GROUP_3.to_list()
    Y_predicted_tot = Y_predicted.PRODUCTION_GROUP_2.to_list() +  Y_predicted.PRODUCTION_GROUP_3.to_list() 
    ErrorTotal = mean_squared_error(Y_train_tot, Y_predicted_tot)
    

    return Y_predicted, ErrorTotal

### 4. Clasificar observaciones, identificando anomalías para cada uno de las series de tiempo de las fabricas

In [9]:
start_time=time.time()
warnings.filterwarnings("ignore") #los warnings omitidos del algoritmo Holt Winters (parece ser que se trata de un BUG)

new_data={}
for ass in range(1,84):
    asset = porAsset.get_group(ass).reset_index().set_index('date')
    new_data[ass] = ClasificacionDeObservaciones(asset)
print("--- %s seconds ---"%(time.time()-start_time))

--- 82.4362382888794 seconds ---


In [10]:
X=pd.concat(new_data.values())
X.to_csv('Output/df_outliers_HoltWinters.csv',index=False)

### 5. Resultados

In [13]:
y_predicted, error= resultados(new_data)
print('error =', np.format_float_scientific(error))
y_predicted

error = 2.454314879484083e+13


,SAMPLE_ID,PRODUCTION_GROUP_2,PRODUCTION_GROUP_3
0,1,4.070000e+06,9.804196e+06
1,2,1.426836e+06,2.945472e+06
2,3,1.247151e+06,2.288254e+06
3,4,1.183898e+06,3.184563e+06
4,5,1.112600e+06,2.475179e+06
...,...,...,...
99,100,1.423158e+06,2.316102e+06
100,101,1.248196e+06,2.232944e+06
101,102,8.912764e+05,2.732699e+06
102,103,1.409321e+06,3.093089e+06
